In [ ]:
# BILIOTECAS
# webscrapy
from urllib.request import Request, urlopen
import json

# data work
import pandas as pd

# graph
import plotly.express as px
import dash
from dash import dcc
from dash import html
import io
from base64 import b64encode

In [ ]:
# DATA 1: GeoJson
with urlopen('https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json') as response:
    geobr = json.load(response)

In [ ]:
# separate WestMiddle
for i in range(len(geobr['features'])-1, -1, -1):
    if geobr['features'][i]['properties']['id'][0:2] not in ['50','51','52','53']:
        geobr['features'].pop(i)

In [ ]:
# DATA 2: COVID BR
req = Request('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)
df_covid = pd.read_csv(content, compression='gzip')

In [ ]:
# treatment and cleaning
df_covid = df_covid.loc[df_covid['state'].isin(['GO','MT','MS','DF'])] # separate WestMiddle
df_covid = df_covid[['city','city_ibge_code','epidemiological_week','new_confirmed']] # select columns
df_covid = df_covid.loc[pd.notna(df_covid['city'])] # excluir nan
df_covid = df_covid.loc[pd.notna(df_covid['city_ibge_code'])] # excluir nan
df_covid['city_ibge_code'] = df_covid['city_ibge_code'].astype('str') # transform type
df_covid['city_ibge_code'] = df_covid['city_ibge_code'].str.replace('\.0','') # clean data
df_covid = df_covid.loc[df_covid['new_confirmed'] >= 0] # clean

In [ ]:
# group by epidemiological_week :: sum days to week
df_covid = df_covid.groupby(['city','city_ibge_code','epidemiological_week'], as_index = False).sum()

# final adjust :: sorting data
df_covid = df_covid.sort_values(by = ['epidemiological_week'], ascending = True)

In [ ]:
# set var buffer 
buffer = io.StringIO()

In [ ]:
# MAP choropleth_mapbox
fig = px.choropleth_mapbox(df_covid, 
                    geojson = geobr, 
                    locations = 'city_ibge_code',
                    featureidkey = "properties.id",
                    color = 'new_confirmed',
                    color_continuous_scale = "OrRd",
                    range_color=(0, 300),
                    animation_frame = 'epidemiological_week',
                    mapbox_style = "carto-positron",
                    zoom = 4.5,
                    center = {"lat":-15.6, "lon": -52.7},
                    opacity = 0.5,
                    labels={'new_confirmed':'Casos novos confirmados'}
                    )

In [ ]:
# set map in buffer
fig.write_html(buffer)

In [ ]:
# serialize buffer e encode
html_bytes = buffer.getvalue().encode()
encoded = b64encode(html_bytes).decode()

# start app dash
app = dash.Dash(__name__)

In [ ]:
# add dash layout
app.layout = html.Figure([
    dcc.Graph(id="graph", figure = fig, style={'width': '1920', 'height': '90vh'})
])

In [ ]:
# run server dash
app.run_server(debug=True, use_reloader=False)